In [1]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from __future__ import print_function
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,11 mins 10 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.10
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_rishav_miiyum
H2O cluster total nodes:,1
H2O cluster free memory:,1.514 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [2]:
df = h2o.import_file(path = "Claims1.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
df.head()

WeekOfMonth,WeekOfMonthClaimed,Age,FraudFound_P,PolicyNumber,RepNumber,Deductible,DriverRating,Year,Month,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,Sex,MaritalStatus,Fault,PolicyType,VehicleCategory,VehiclePrice,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,BasePolicy
5,1,21,0,1,12,300,1,1994,11,2,6,1,1,0,0,0,0,5,1,5,4,3,0,2,3,1,0,1,0,1,2,2
3,4,34,0,2,15,400,4,1994,0,2,6,1,0,0,1,0,0,4,1,5,4,3,0,5,4,0,0,1,0,0,0,1
5,2,47,0,3,7,400,3,1994,9,4,6,1,3,10,1,1,0,4,1,5,4,3,1,6,6,1,0,1,0,0,0,1
2,1,65,0,4,4,400,2,1994,6,5,17,0,4,6,1,1,1,2,1,1,4,3,1,7,7,0,0,1,3,0,0,2
5,2,27,0,5,3,400,1,1994,0,0,6,1,1,1,0,0,1,4,1,5,4,3,0,4,4,1,0,1,0,0,0,1
4,1,20,0,6,12,400,3,1994,9,4,6,1,2,10,1,0,1,4,1,5,4,3,0,4,2,1,0,1,2,0,0,1
1,3,36,0,7,14,400,1,1994,1,5,6,1,0,1,1,1,1,4,1,5,4,3,1,6,5,1,0,1,1,0,0,1
1,4,0,0,8,1,400,4,1994,10,4,6,1,1,2,1,0,0,4,1,5,4,3,1,0,0,1,0,1,0,0,0,1
4,5,30,0,9,7,400,4,1994,11,5,6,1,2,11,1,0,0,4,1,5,4,3,0,5,4,1,1,1,2,0,0,1
3,3,42,0,10,7,400,1,1994,3,1,5,1,2,3,1,1,0,6,2,5,4,3,2,7,5,1,0,1,2,0,0,0


In [4]:
cols = list(df.columns)
cols.pop(cols.index('FraudFound_P'))
df = df[cols+['FraudFound_P']]

In [5]:
df[32] = df[32].asfactor()
predictors = df.columns[0:32]
response = 'FraudFound_P'

In [6]:
df.head()

WeekOfMonth,WeekOfMonthClaimed,Age,PolicyNumber,RepNumber,Deductible,DriverRating,Year,Month,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,Sex,MaritalStatus,Fault,PolicyType,VehicleCategory,VehiclePrice,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,BasePolicy,FraudFound_P
5,1,21,1,12,300,1,1994,11,2,6,1,1,0,0,0,0,5,1,5,4,3,0,2,3,1,0,1,0,1,2,2,0
3,4,34,2,15,400,4,1994,0,2,6,1,0,0,1,0,0,4,1,5,4,3,0,5,4,0,0,1,0,0,0,1,0
5,2,47,3,7,400,3,1994,9,4,6,1,3,10,1,1,0,4,1,5,4,3,1,6,6,1,0,1,0,0,0,1,0
2,1,65,4,4,400,2,1994,6,5,17,0,4,6,1,1,1,2,1,1,4,3,1,7,7,0,0,1,3,0,0,2,0
5,2,27,5,3,400,1,1994,0,0,6,1,1,1,0,0,1,4,1,5,4,3,0,4,4,1,0,1,0,0,0,1,0
4,1,20,6,12,400,3,1994,9,4,6,1,2,10,1,0,1,4,1,5,4,3,0,4,2,1,0,1,2,0,0,1,0
1,3,36,7,14,400,1,1994,1,5,6,1,0,1,1,1,1,4,1,5,4,3,1,6,5,1,0,1,1,0,0,1,0
1,4,0,8,1,400,4,1994,10,4,6,1,1,2,1,0,0,4,1,5,4,3,1,0,0,1,0,1,0,0,0,1,0
4,5,30,9,7,400,4,1994,11,5,6,1,2,11,1,0,0,4,1,5,4,3,0,5,4,1,1,1,2,0,0,1,0
3,3,42,10,7,400,1,1994,3,1,5,1,2,3,1,1,0,6,2,5,4,3,2,7,5,1,0,1,2,0,0,0,0


In [7]:
train, test, valid = df.split_frame(ratios=[.7, .15], seed = 1234)

In [8]:
nfolds = 5

In [9]:
my_gbm = H2OGradientBoostingEstimator(balance_classes = True,
                                      distribution="bernoulli",
                                      ntrees=10,
                                      max_depth=3,
                                      min_rows=2,
                                      learn_rate=0.2,
                                      nfolds=nfolds,
                                      fold_assignment="Modulo",
                                      keep_cross_validation_predictions=True,
                                      seed=1234)

In [10]:
my_gbm.train(x=predictors, y=response, training_frame=train, validation_frame = valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [11]:
my_rf = H2ORandomForestEstimator(balance_classes = True,
                                 ntrees=50,
                                 nfolds=nfolds,
                                 fold_assignment="Modulo",
                                 keep_cross_validation_predictions=True,
                                 seed=1234)

In [12]:
my_rf.train(x=predictors, y=response, training_frame=train, validation_frame = valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [13]:
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_binomial",
                                       base_models=[my_gbm, my_rf])
ensemble.train(x=predictors, y=response, training_frame=train)

stackedensemble Model Build progress: |███████████████████████████████████| 100%


In [14]:
perf_stack_test = ensemble.model_performance(test)

In [15]:
perf_gbm_test = my_gbm.model_performance(test)
perf_rf_test = my_rf.model_performance(test)
baselearner_best_auc_test = max(perf_gbm_test.auc(), perf_rf_test.auc())
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

Best Base-learner Test AUC:  0.8873292558613659
Ensemble Test AUC:  0.8790655793408086


In [16]:
pred = ensemble.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%
